# 3.2. Implementación de regresión lineal desde cero

Ahora que comprende las ideas clave detrás de la regresión lineal, podemos comenzar a trabajar a través de una implementación práctica en el código. En esta sección, implementaremos todo el método desde cero, incluida la canalización de datos, el modelo, la función de pérdida y el optimizador de descenso de gradiente estocástico de minilotes. Si bien los marcos modernos de aprendizaje profundo pueden automatizar casi todo este trabajo, implementar cosas desde cero es la única forma de asegurarse de que realmente sabe lo que está haciendo. Además, cuando llegue el momento de personalizar los modelos, definiendo nuestras propias capas o funciones de pérdida, será útil comprender cómo funcionan las cosas debajo del capó. En esta sección, nos basaremos únicamente en tensores y diferenciación automática. Luego, presentaremos una implementación más concisa, aprovechando las ventajas de los marcos de trabajo de aprendizaje profundo.

In [94]:
use strict;
use warnings;
use Data::Dump qw(dump);
use Data::Dumper qw(Dumper);
use AI::MXNet qw(mx);
use List::Util qw(shuffle);
use List::Util qw(min);
use d2l;
IPerl->load_plugin('Chart::Plotly');
use Chart::Plotly::Plot

# 3.2.1. Generación del conjunto de datos
Para simplificar las cosas, construiremos un conjunto de datos artificial de acuerdo con un modelo lineal con ruido aditivo. Nuestra tarea será recuperar los parámetros de este modelo utilizando el conjunto finito de ejemplos contenidos en nuestro conjunto de datos. Mantendremos los datos de baja dimensión para que podamos visualizarlos fácilmente. En el siguiente fragmento de código, generamos un conjunto de datos que contiene 1000 ejemplos, cada uno de los cuales consta de 2 características muestreadas de una distribución normal estándar. Por lo tanto, nuestro conjunto de datos sintéticos será una matriz X∈R1000×2.

Los verdaderos parámetros que generarán nuestro conjunto de datos serán w=[2,−3.4]⊤ y b=4.2 , y nuestras etiquetas sintéticas se asignarán de acuerdo con el siguiente modelo lineal con el término de ruido ϵ :

(3.2.1)
y=Xw+b+ϵ.
 
Podría pensar en ϵ como capturando posibles errores de medición en las características y etiquetas. Supondremos que se cumplen los supuestos estándar y, por lo tanto, que ϵ obedece a una distribución normal con media de 0. Para simplificar nuestro problema, estableceremos su desviación estándar en 0.01. El siguiente código genera nuestro conjunto de datos sintéticos.

In [83]:
sub synthetic_data {
    my ($w, $b, $num_examples) = @_;
    my $X = mx->nd->random->normal(0, 1, [$num_examples, scalar(@{$w})]);
    my $y = mx->nd->dot($X, $w) + $b;
    $y += mx->nd->random->normal(0, 0.01, $y->shape);
    return ($X, $y->reshape([-1, 1]));
}

my $true_w = mx->nd->array([2, -3.4]);
my $true_b = 4.2;
my ($features, $labels) = synthetic_data($true_w, $true_b, 10);

<AI::MXNet::NDArray 10x2 @cpu(0)><AI::MXNet::NDArray 10x1 @cpu(0)>

Warning: Subroutine synthetic_data redefined at reply input line 1.


Note that each row in features consists of a 2-dimensional data example and that each row in labels consists of a 1-dimensional label value (a scalar).

In [84]:
print dump $features->asarray;
print dump $labels->asarray;

[
  [1.26625561714172, 0.895064175128937],
  [-0.601594507694244, 1.2040559053421],
  [-0.971219301223755, -0.582562267780304],
  [0.371707707643509, 0.930007219314575],
  [-1.4225754737854, -0.517619907855988],
  [2.0088324546814, 0.286308497190475],
  [0.560459494590759, 0.96975976228714],
  [-0.528537452220917, -1.88908994197845],
  [0.654791235923767, -0.454813152551651],
  [0.325108081102371, -1.30023407936096],
][
  [3.69297194480896],
  [-1.08244490623474],
  [4.24068832397461],
  [1.78618025779724],
  [3.12444519996643],
  [7.23399782180786],
  [2.02305436134338],
  [9.56264305114746],
  [7.05418395996094],
  [9.27457714080811],
]

1

By generating a scatter plot using the second feature features[:, 1] and labels, we can clearly observe the linear correlation between the two.
 

In [85]:



print dump ('features:', $features->slice(1,[0,1])->asarray);
print "\n";
print dump "labels:", $labels->slice(1,[0,0])->asarray;


("features:", [[-0.601594507694244, 1.2040559053421]])
("labels:", [[-1.08244490623474]])

1

In [125]:
print dump d2l->set_figsize();
print dump  d2l->plt->scatter(features('X', 0)->asarray(), labels->asarray(), 1);


Error: Can't locate object method "set_figsize" via package ""d2l"" (perhaps you forgot to load ""d2l""?) at reply input line 1.


In [123]:

# Obtiene los datos de las características y las etiquetas
my $x_data = $features->asarray;
my $y_data = $labels->asarray;
# Crea el objeto de gráfico
my %parametros = (
    x => $x_data,
    y => $y_data,
    type => 'scatter',
    mode => 'markers',
    marker => {
        size => 6,
        color => 'rgb(0, 0, 255)',
    },
);
my $chart = Chart::Plotly::Plot->new(%parametros);

# Gráfica los datos
$chart->show();

Error: Can't locate object method "show" via package "Chart::Plotly::Plot" at reply input line 19.


# 3.2.2. Lectura del conjunto de datos
Recuerde que los modelos de entrenamiento consisten en realizar varias pasadas sobre el conjunto de datos, tomando un minilote de ejemplos a la vez y usándolos para actualizar nuestro modelo. Dado que este proceso es tan fundamental para entrenar algoritmos de aprendizaje automático, vale la pena definir una función de utilidad para mezclar el conjunto de datos y acceder a él en minilotes.

En el siguiente código, definimos la función data_iter para demostrar una posible implementación de esta funcionalidad. La función toma un tamaño de lote, una matriz de características y un vector de etiquetas, lo que genera minilotes del tamaño tamaño_lote. Cada minilote consta de una tupla de características y etiquetas.

In [105]:
sub data_iter {
    my ($batch_size, $features, $labels) = @_;
    my $num_examples = $features->shape->[0];
    my @indices = (0 .. $num_examples - 1);

    List::Util::shuffle(@indices);
    for (my $i = 0; $i < $num_examples; $i += $batch_size) {
        my @batch_indices = @indices[$i .. List::Util::min($i + $batch_size - 1, $num_examples - 1)];
        my $batch_indices = mx->array(\@batch_indices);
        yield $features->index_axis(1, $batch_indices), $labels->index_axis(1, $batch_indices);
    }
}


Warning: Subroutine data_iter redefined at reply input line 1.


En general, tenga en cuenta que queremos usar minilotes de tamaño razonable para aprovechar el hardware de la GPU, que sobresale en las operaciones de paralelización. Debido a que cada ejemplo se puede alimentar a través de nuestros modelos en paralelo y el gradiente de la función de pérdida para cada ejemplo también se puede tomar en paralelo, las GPU nos permiten procesar cientos de ejemplos en apenas más tiempo del que se necesitaría para procesar un solo ejemplo. .

Para desarrollar algo de intuición, leamos e imprimamos el primer lote pequeño de ejemplos de datos. La forma de las características en cada minilote nos indica tanto el tamaño del minilote como la cantidad de características de entrada. Asimismo, nuestro minilote de etiquetas tendrá una forma dada por lote_tamaño.

In [137]:
my $batch_size = 10;
my $iter = data_iter($batch_size, $features, $labels);
while (my ($X, $y) = $iter->()) {
    print Dumper ($X), "\n", Dumper ($y);
    last;
}

Warning: Can't locate object method "array" via package "AI::MXNet" at reply input line 9.

Use of uninitialized value in goto at (eval 952) line 9.


Error: goto must have label at (eval 952) line 9.


A medida que ejecutamos la iteración, obtenemos distintos minilotes sucesivamente hasta que se agota todo el conjunto de datos (pruebe esto). Si bien la iteración implementada anteriormente es buena para fines didácticos, es ineficiente en formas que podrían causarnos problemas en problemas reales. Por ejemplo, requiere que carguemos todos los datos en la memoria y que realicemos muchos accesos aleatorios a la memoria. Los iteradores incorporados implementados en un marco de aprendizaje profundo son considerablemente más eficientes y pueden manejar tanto datos almacenados en archivos como datos alimentados a través de flujos de datos.

# 3.2.3. Inicialización de los parámetros del modelo
Antes de que podamos comenzar a optimizar los parámetros de nuestro modelo mediante el descenso de gradiente estocástico de minilotes, necesitamos tener algunos parámetros en primer lugar. En el siguiente código, inicializamos los pesos mediante el muestreo de números aleatorios de una distribución normal con media 0 y una desviación estándar de 0,01, y estableciendo el sesgo en 0.

In [54]:
my $w = mx->random->normal(0, 0.01, [2, 1]);
my $b = mx->zeros([1]);
$w->attach_grad();
$b->attach_grad();


Warning: Can't locate object method "zeros" via package "AI::MXNet" at reply input line 2.

Use of uninitialized value in goto at (eval 952) line 9.


Error: goto must have label at (eval 952) line 9.


Después de inicializar nuestros parámetros, nuestra siguiente tarea es actualizarlos hasta que se ajusten lo suficientemente bien a nuestros datos. Cada actualización requiere tomar el gradiente de nuestra función de pérdida con respecto a los parámetros. Dado este gradiente, podemos actualizar cada parámetro en la dirección que pueda reducir la pérdida.

Dado que nadie quiere calcular gradientes explícitamente (esto es tedioso y propenso a errores), usamos la diferenciación automática, como se presentó en la Sección 2.5, para calcular el gradiente.

# 3.2.4. Definición del modelo
A continuación, debemos definir nuestro modelo, relacionando sus entradas y parámetros con sus salidas. Recuerde que para calcular la salida del modelo lineal, simplemente tomamos el producto escalar matriz-vector de las características de entrada X y los pesos del modelo w, y agregamos el desplazamiento b a cada ejemplo. Tenga en cuenta que debajo Xw es un vector y b es un escalar. Recuerde el mecanismo de transmisión como se describe en la Sección 2.1.3. Cuando sumamos un vector y un escalar, el escalar se suma a cada componente del vector.

In [55]:
sub linreg {
    my ($X, $w, $b) = @_;
    return mx->dot($X, $w) + $b;
}


# 3.2.5. Definición de la función de pérdida
Dado que actualizar nuestro modelo requiere tomar el gradiente de nuestra función de pérdida, primero debemos definir la función de pérdida. Aquí usaremos la función de pérdida al cuadrado como se describe en la Sección 3.1. En la implementación, necesitamos transformar el valor real y en la forma y_hat del valor predicho. El resultado devuelto por la siguiente función también tendrá la misma forma que y_hat.

In [44]:
sub squared_loss {
    my ($y_hat, $y) = @_;
    return ($y_hat - $y->reshape($y_hat->shape)) ** 2 / 2;
}


# 3.2.6. Definición del algoritmo de optimización
Como discutimos en la Sección 3.1, la regresión lineal tiene una solución de forma cerrada. Sin embargo, este no es un libro sobre regresión lineal: es un libro sobre aprendizaje profundo. Dado que ninguno de los otros modelos que presenta este libro se puede resolver analíticamente, aprovecharemos esta oportunidad para presentar su primer ejemplo práctico de descenso de gradiente estocástico de minilotes.

En cada paso, utilizando un minilote extraído aleatoriamente de nuestro conjunto de datos, estimaremos el gradiente de pérdida con respecto a nuestros parámetros. A continuación, actualizaremos nuestros parámetros en la dirección que pueda reducir la pérdida. El siguiente código aplica la actualización de descenso de gradiente estocástico de minilotes, dado un conjunto de parámetros, una tasa de aprendizaje y un tamaño de lote. El tamaño del paso de actualización está determinado por la tasa de aprendizaje lr. Debido a que nuestra pérdida se calcula como una suma sobre el minilote de ejemplos, normalizamos nuestro tamaño de paso por el tamaño del lote (batch_size), de modo que la magnitud de un tamaño de paso típico no dependa en gran medida de nuestra elección del tamaño del lote.

In [35]:
sub sgd {
    my ($params, $lr, $batch_size) = @_;
    for my $param (@{$params}) {
        $param->slice(':') .= $param - ($lr * $param->grad / $batch_size);
    }
}


# 3.2.7. Capacitación
Ahora que tenemos todas las partes en su lugar, estamos listos para implementar el ciclo de entrenamiento principal. Es crucial que comprenda este código porque verá bucles de entrenamiento casi idénticos una y otra vez a lo largo de su carrera en el aprendizaje profundo.

En cada iteración, tomaremos un minilote de ejemplos de entrenamiento y los pasaremos a través de nuestro modelo para obtener un conjunto de predicciones. Después de calcular la pérdida, iniciamos el paso hacia atrás por la red, almacenando los gradientes con respecto a cada parámetro. Finalmente, llamaremos al algoritmo de optimización sgd para actualizar los parámetros del modelo.

En resumen, ejecutaremos el siguiente bucle:

Inicializar parámetros (w,b)

Repetir hasta terminar

Calcular el gradiente g←∂(w,b)1|B|∑i∈Bl(x(i),y(i),w,b)

Actualizar parámetros (w,b)←(w,b)−ηg

En cada época, iteraremos a través de todo el conjunto de datos (usando la función data_iter) una vez que pasemos por cada ejemplo en el conjunto de datos de entrenamiento (asumiendo que la cantidad de ejemplos es divisible por el tamaño del lote). El número de épocas num_epochs y la tasa de aprendizaje lr son ambos hiperparámetros, que establecemos aquí en 3 y 0,03, respectivamente. Desafortunadamente, establecer hiperparámetros es complicado y requiere algunos ajustes por ensayo y error. Omitimos estos detalles por ahora, pero los revisamos más adelante en la Sección 11.

In [36]:
my $lr = 0.03;
my $num_epochs = 3;
my $net = \&linreg;
my $loss = \&squared_loss;

for (my $epoch = 0; $epoch < $num_epochs; $epoch++) {
    my $iter = data_iter($batch_size, $features, $labels);
    while (my ($X, $y) = $iter->()) {
        Autograd->record(sub {
            my $l = $loss->($net->($X, $w, $b), $y);
            $l->backward();
        });
        sgd([$w, $b], $lr, $batch_size);
    }
    my $train_l = $loss->($net->($features, $w, $b), $labels);
    printf("epoch %d, loss %f", $epoch + 1, $train_l->mean);
}


Error: Can't call method "shape" on unblessed reference at reply input line 3.


En este caso, debido a que nosotros mismos sintetizamos el conjunto de datos, sabemos con precisión cuáles son los parámetros verdaderos. Por lo tanto, podemos evaluar nuestro éxito en el entrenamiento comparando los parámetros reales con los que aprendimos a través de nuestro ciclo de entrenamiento. De hecho, resultan estar muy cerca uno del otro.

In [37]:
printf("error in estimating w: %s\n", $true_w - $w->reshape($true_w->shape));
printf("error in estimating b: %s\n", $true_b - $b);


Error: Can't call method "shape" on an undefined value at reply input line 1.


Tenga en cuenta que no debemos dar por sentado que podemos recuperar los parámetros perfectamente. Sin embargo, en el aprendizaje automático, normalmente nos preocupamos menos por recuperar los parámetros subyacentes reales y más por los parámetros que conducen a una predicción muy precisa. Afortunadamente, incluso en problemas de optimización difíciles, el descenso de gradiente estocástico a menudo puede encontrar soluciones muy buenas, debido en parte al hecho de que, para redes profundas, existen muchas configuraciones de los parámetros que conducen a una predicción muy precisa.

# 3.2.8. Resumen
Vimos cómo se puede implementar y optimizar una red profunda desde cero, utilizando solo tensores y diferenciación automática, sin necesidad de definir capas ni optimizadores sofisticados.

Esta sección solo rasca la superficie de lo que es posible. En las siguientes secciones, describiremos modelos adicionales basados ​​en los conceptos que acabamos de presentar y aprenderemos a implementarlos de manera más concisa.